In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import regex as re
import plotly.graph_objects as go
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from joblib import dump, load
import lightgbm as lgb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/CS313/Data

/content/drive/.shortcut-targets-by-id/1kxVDaxTg_LxLJEdgUVudHmlG01RBmQaU/CS313/Data


## function

In [ ]:
def normalize_string(s):
  try:
    if s.endswith('\'') & s.startswith(' \''):
      return s[2:-1]
    return s
  except:
    return s

def replace_null(x):
  try:
    if "NULL" in x:
        return np.nan
    else:
        return x
  except:
    return x

uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"


def loaddicchar():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def covert_unicode(txt):
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

# Preprocess

## sinhvien

In [ ]:
df_student=pd.read_excel('01.sinhvien.xlsx')
df_student['mssv']= df_student['mssv'].str.strip()

df_student = df_student.dropna(how='all')
df_student.columns = df_student.columns.str.lstrip()
df_student=df_student.T.drop_duplicates().T

object_columns = df_student.select_dtypes(include=['object']).columns.tolist()
df_student[object_columns]=df_student.select_dtypes(include=['object']).applymap(normalize_string)

# Áp dụng hàm trên cho toàn bộ DataFrame
object_columns = df_student.select_dtypes(include=['object']).columns.tolist()
df_student[object_columns]=df_student.select_dtypes(include=['object']).applymap(replace_null)

In [ ]:
khoahoc_map={'8': '2013',
             '9': '2014',
             '10':'2015',
             '11':'2016',
             '12':'2017',
             '13':'2018',
             '14':'2019'}
df_student['namkhoahoc']=df_student['khoahoc'].astype('str').map(khoahoc_map)

In [ ]:
df_student

,id,mssv,namsinh,gioitinh,noisinh,lopsh,khoa,hedt,khoahoc,chuyennganh2,tinhtrang,diachi_tinhtp,namkhoahoc
0,1,BE375BAAXPvAibaEXe9JDlHA4z2GHJ3/PVStCxR2,1995.0,1,TP. Hồ Chí Minh,KTPM0001,CNPM,CQUI,8,D480103,3,Thành phố Hồ Chí Minh,2013
1,2,2420ED57XPvAibaEXe/Lh6v1XxTKJa6JLFRUPkLM,1995.0,1,Đồng Tháp,HTTT0001,HTTT,CTTT,8,D480104,3,Huyện Hóc Môn,2013
2,3,83B76C01XPvAibaEXe/lOccskaOiO2K46r7t4qnt,1994.0,1,Hà Nam Ninh,KHMT2013,KHMT,CQUI,8,D480101,5,Tỉnh Hà Nam,2013
3,4,91F785ABXPvAibaEXe/lOccskaOiO5y4GbVvuRQu,1995.0,1,TP. Hồ Chí Minh,HTTT0001,HTTT,CTTT,8,D480104,3,Thành phố Hồ Chí Minh,2013
4,5,007C275DXPvAibaEXe+TFgEDwYNnveOOmOYeYzF6,1995.0,1,Thành phố Hồ Chí Minh,MMTT0001,MMT&TT,CQUI,8,D480201,8,Thành phố Hồ Chí Minh,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8290,19283,7D7633B0XPvAibaEXe95IhrkYSGfcCQVGm4nFGyt,2000.0,1,An Giang,MMCL2019.2,MMT&TT,CLC,14,7480102,2,Tỉnh An Giang,2019
8291,19284,AB431338XPvAibaEXe8xMCTZ03/BexWzDCyWVsT3,2001.0,1,Bình Thuận,KHMT2019,KHMT,CQUI,14,D480101,1,Tỉnh Bình Thuận,2019
8292,19285,75AD7B4AXPvAibaEXe8xMCTZ03/BewoGrWSt0ZXM,2001.0,1,Lâm Đồng,KHMT2019,KHMT,CQUI,14,D480101,1,Tỉnh Lâm Đồng,2019
8293,19345,CB263C18XPvAibaEXe8xMCTZ03/Be8yk40QWdPiR,2000.0,1,Bến Tre,CNTT2019,KTTT,CQUI,14,D480201,1,Tỉnh Bến Tre,2019


In [ ]:
df_student.to_excel('Preprocessed/01.sinhvien_preprocessed.xlsx', index=False)

## XLHV

In [ ]:
df_XLHV= pd.read_excel('08.XLHV.xlsx')
df_XLHV.columns = df_XLHV.columns.str.lstrip()

#  dịch các cột phía sau
for i in range(len(df_XLHV)):
    if df_XLHV.loc[i, "Column1"] and pd.isna(df_XLHV.loc[i, "hocky"]):
        df_XLHV.loc[i, "hocky":] = df_XLHV.loc[i, "hocky":].shift(-1)

df_XLHV = df_XLHV.drop("Column1", axis=1)
df_XLHV = df_XLHV.dropna(how='all')
df_XLHV["namhoc"] = df_XLHV["namhoc"].astype(float)
df_XLHV['hk']=df_XLHV['hocky']+(df_XLHV['namhoc']-1)*2

object_columns = df_XLHV.select_dtypes(include=['object']).columns.tolist()
df_XLHV[object_columns]=df_XLHV.select_dtypes(include=['object']).applymap(normalize_string)

In [ ]:
#xử lý và gom các cụm theo lý do
df_XLHV['lydo'] = df_XLHV['lydo'].str.lstrip().astype('string')
df_XLHV['lydo'] = df_XLHV['lydo'].apply(lambda x: x.replace(':\\n-', '').replace('\\n-', ''))
df_XLHV['lydo'] = df_XLHV['lydo'].apply(lambda x: x.replace(':', ''))
df_XLHV['lydo'] = df_XLHV['lydo'].apply(lambda x: x.replace('trể', 'trễ'))

df_XLHV['lydo'] = df_XLHV['lydo'].replace(r'\s+', ' ', regex=True)
df_XLHV['lydo'] = df_XLHV['lydo'].str.replace("^'", '', regex=True)
df_XLHV['lydo'] = df_XLHV['lydo'].str.replace('báo', 'cáo', regex=True)
df_XLHV['lydo'] = df_XLHV['lydo'].str.replace('Bị cảnh cáo vì đtb 2 học kỳ tiếp < 4', 'Bị cảnh cáo vì đtb 2 học kỳ liên tiếp < 4', regex=True)
df_XLHV['lydo'] = df_XLHV['lydo'].str.replace('Buộc thôi học', 'BTH', regex=True)
df_XLHV['lydo'] = df_XLHV['lydo'].str.capitalize()
df_XLHV['lydo'] = df_XLHV['lydo'].str.replace('điểm trung bình', 'đtb', regex=True)

In [ ]:
def check_string(string):
    if ('đóng học phí trễ' not in string) and (('Bị cảnh cáo' in string) and ('đtb học kỳ' in string)):
        return 'Bị cảnh cáo vì ĐTB học kỳ'
    elif ('Bị buộc thôi học vì đtb' in string) or (('Bth' in string) and ('đtb' in string)):
        return 'Buộc thôi học vì ĐTB học kỳ'
    elif ('đóng học phí trễ'in string) and ('Bị cảnh cáo vì đtb học kỳ' in string):
        return 'Bị cảnh cáo vì ĐTB và trễ học phí'
    elif 'quá hạn đào tạo' in string:
        return 'Quá hạn đào tạo'
    elif 'xem xét hạ mức' in string:
        return 'Được xem xét hạ mức'
    elif ('bảo lưu' in string) and ('không nhập học lại' in string) or ('Bảo lưu nhưng không nhập học lại' in string) or ('Bảo lưu không nhập học lại'in string):
        return 'Buộc thôi học vì bảo lưu không nhập học lại'
    elif ('lần thứ 3 liên tiếp' in string) or ('3 lần liên tiếp' in string) or ('3 hk liên tiếp'in string):
        return "Buộc thôi học vì cảnh báo 3 lần liên tiếp"
    elif ('chuyển tự do' in string)or ('chuyển từ xa' in string) or('sang hệ từ xa' in string):
        return 'Xem xét chuyển qua hệ đào tạo không chính quy' 
    elif ('đóng học phí trễ'in string) or 'đóng học phí trễ' in string:
        return 'Bị cảnh cáo vì đóng học phí trễ'
    elif 'Bị cảnh cáo vì đtb 2 học kỳ tiếp < 4' in string:
        return('Bị cảnh cáo vì đtb 2 học kỳ liên tiếp < 4')
    else:
        return string

df_XLHV['lydo'] = df_XLHV['lydo'].astype('str').apply(check_string) 

In [ ]:
# Thay đổi giá trị trong cột 'trangthai'
df_XLHV.loc[df_XLHV['lydo'].str.contains('Buộc thôi học'), 'tinhtrang'] = 5
df_XLHV.loc[df_XLHV['lydo'].str.contains('Bị cảnh cáo'), 'tinhtrang'] = 2

In [ ]:
df_XLHV.head(10)

,id,mssv,tinhtrang,lydo,hocky,namhoc,soqd,ngayqd,hk
0,7595.0,98247E19XPvAibaEXe93PEySAJOVkxsz+HIviyYJ,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
1,7596.0,52B1CC2DXPvAibaEXe+kwIr/VXjl0w+FyuW1abQ7,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
2,7597.0,5666EAF4XPvAibaEXe+Z7KfNHYLttJaLI+SH+DpO,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
3,7598.0,682C32E0XPvAibaEXe/Bn5cQSu92WX0DcIKdud8D,2.0,Bị cảnh cáo vì đtb 2 học kỳ liên tiếp < 4,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
4,7599.0,7901D3A9XPvAibaEXe+6MN9FoW2mqYEmruXXBNsS,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
5,7600.0,82D67FF0XPvAibaEXe9Bx5CZuIA9z+ymYnRBj6Fr,2.0,Bị cảnh cáo vì ĐTB và trễ học phí,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
6,7601.0,FBD9AAFEXPvAibaEXe+5oCRQPDKXsjh0OTYpdD2Y,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
7,7602.0,B47BC5E8XPvAibaEXe/H1VZJBzJAwmY5aGwFIvk2,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
8,7603.0,7FBD0513XPvAibaEXe+ltCXpm838vaXT1ZHB2inA,2.0,Bị cảnh cáo vì đóng học phí trễ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0
9,7604.0,C3E0BACEXPvAibaEXe8zBeLfIMmoBC4sDuuDh1j3,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,2016.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4032.0


In [ ]:
df_XLHV['lydo'].value_counts()

Bị cảnh cáo vì ĐTB học kỳ                        1417
Buộc thôi học vì ĐTB học kỳ                       693
Bị cảnh cáo vì đóng học phí trễ                   337
Được xem xét hạ mức                               314
Bị cảnh cáo vì ĐTB và trễ học phí                 171
Xem xét chuyển qua hệ đào tạo không chính quy     153
Bị cảnh cáo vì đtb 2 học kỳ liên tiếp < 4         113
Buộc thôi học vì cảnh báo 3 lần liên tiếp         101
Buộc thôi học vì bảo lưu không nhập học lại        87
Quá hạn đào tạo                                    59
Name: lydo, dtype: int64

In [ ]:
df_XLHV.to_excel('Preprocessed/08.XLHV_preprocessed.xlsx', index=False)

### xử lý học vụ

In [ ]:
df_warning=df_XLHV[df_XLHV['tinhtrang']==2]

In [ ]:
df_warning=df_warning.merge(df_student[['mssv','namkhoahoc']], on='mssv')
df_warning['namhoc']=df_warning['namhoc']-df_warning['namkhoahoc'].astype('int')+1
df_warning['hk']=df_warning['hocky']+(df_warning['namhoc']-1)*2
df_warning['lanthu']=df_warning.sort_values(['mssv','hk'],ascending=True).groupby('mssv').cumcount()+1

In [ ]:
df_warning.head(3)

,id,mssv,tinhtrang,lydo,hocky,namhoc,soqd,ngayqd,hk,namkhoahoc,lanthu
0,7595.0,98247E19XPvAibaEXe93PEySAJOVkxsz+HIviyYJ,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,4.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,8.0,2013,1
1,9457.0,98247E19XPvAibaEXe93PEySAJOVkxsz+HIviyYJ,2.0,Bị cảnh cáo vì ĐTB học kỳ,2.0,5.0,180/QĐ-ĐHCNTT,2018-03-29 00:00:00,10.0,2013,2
2,10367.0,98247E19XPvAibaEXe93PEySAJOVkxsz+HIviyYJ,2.0,Bị cảnh cáo vì ĐTB học kỳ,1.0,6.0,NaT,2018-10-12,11.0,2013,3


In [ ]:
hk_xlhv = df_XLHV[df_XLHV['tinhtrang'].isin([5,7,8])][['mssv','hk']]

for indx, row in hk_xlhv.iterrows():
  df_warning=df_warning.drop(df_warning.loc[((df_warning['hk'] >= row['hk']) & (df_warning['mssv']== row['mssv']))].index)

df_warning=df_warning.drop(df_warning.loc[(df_warning['hk'] >12)].index)

In [ ]:
df_warning['lydo'].value_counts()

Bị cảnh cáo vì ĐTB học kỳ                    1404
Bị cảnh cáo vì đóng học phí trễ               334
Được xem xét hạ mức                           300
Bị cảnh cáo vì ĐTB và trễ học phí             168
Bị cảnh cáo vì đtb 2 học kỳ liên tiếp < 4     113
Name: lydo, dtype: int64

In [ ]:
pivot_df_warning=df_warning[['mssv','lanthu','lydo']].pivot_table(index='mssv', columns='lanthu', values='lydo', aggfunc='first')

new_columns=[]
for column in pivot_df_warning.columns:
    lanthu = column
    new_column = f"CCHV_{lanthu}"
    new_columns.append(new_column)

pivot_df_warning.columns= new_columns
pivot_df_warning.reset_index(inplace=True)
pivot_df_warning.head(3)

,mssv,CCHV_1,CCHV_2,CCHV_3,CCHV_4,CCHV_5,CCHV_6
0,001E045BXPvAibaEXe+n07P56kWx2N6EoOCUJBA4,Bị cảnh cáo vì ĐTB học kỳ,Bị cảnh cáo vì ĐTB học kỳ,NaN,NaN,NaN,NaN
1,004E12B9XPvAibaEXe81r7KrCH0uc0uxBtoBw4BG,Bị cảnh cáo vì đtb 2 học kỳ liên tiếp < 4,Bị cảnh cáo vì ĐTB học kỳ,Bị cảnh cáo vì ĐTB học kỳ,NaN,NaN,NaN
2,005116E7XPvAibaEXe8VdV75E6kcrk07Zn1wf/T8,Bị cảnh cáo vì đóng học phí trễ,Bị cảnh cáo vì ĐTB học kỳ,NaN,NaN,NaN,NaN


In [ ]:
namkhoahoc= list(df_warning['namkhoahoc'].value_counts().index)

In [ ]:
df_warning.to_excel('Preprocessed/CCHV_preprocessed.xlsx', index=False)

## Buoc thoi hoc (label 1)

In [ ]:
df_BTH=df_XLHV[df_XLHV['tinhtrang'].isin([5,7,8])]
df_BTH=df_BTH.merge(df_student[['mssv','namkhoahoc']], on='mssv')

In [ ]:
df_BTH.value_counts('lydo')

lydo
Buộc thôi học vì ĐTB học kỳ                      693
Xem xét chuyển qua hệ đào tạo không chính quy    153
Buộc thôi học vì cảnh báo 3 lần liên tiếp        101
Buộc thôi học vì bảo lưu không nhập học lại       87
Quá hạn đào tạo                                   59
dtype: int64

In [ ]:
df_BTH.to_excel('Preprocessed/BTH_preprocessed.xlsx', index=False)

In [ ]:
df_BTH['label']=1

In [ ]:
# train_pos, test_pos = train_test_split(df_BTH[['mssv','label']], test_size=1 - train_ratio)

# val_pos, test_pos = train_test_split(test_pos, test_size=test_ratio/(test_ratio + validation_ratio)) 
train_pos, test_pos = train_test_split(df_BTH[['mssv','label']], train_size=0.7, random_state=18)

In [ ]:
df_BTH['namhoc']=df_BTH['namhoc']-df_BTH['namkhoahoc'].astype('int')+1
df_BTH['hk']=df_BTH['hocky']+(df_BTH['namhoc']-1)*2

In [ ]:
df_BTH

,id,mssv,tinhtrang,lydo,hocky,namhoc,soqd,ngayqd,hk,namkhoahoc,label
0,7859.0,667D7118XPvAibaEXe9fafwYRU+Npl+YtdtCh0Qt,5.0,Buộc thôi học vì ĐTB học kỳ,2.0,4.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,8.0,2013,1
1,7860.0,C78857DAXPvAibaEXe+UuZ4QW1XoS0CuT9CeY9Z/,5.0,Buộc thôi học vì ĐTB học kỳ,2.0,3.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,6.0,2014,1
2,7861.0,C1732776XPvAibaEXe+UuZ4QW1XoS0uZyqgzu4FR,5.0,Buộc thôi học vì cảnh báo 3 lần liên tiếp,2.0,3.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,6.0,2014,1
3,7862.0,6D3C6B7CXPvAibaEXe+RaLErOlzghYXwH+RIzcAy,5.0,Buộc thôi học vì ĐTB học kỳ,2.0,2.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,4.0,2015,1
4,7869.0,DEC271BCXPvAibaEXe8WD3qnVG2b/XsMiSHc1tc+,5.0,Buộc thôi học vì ĐTB học kỳ,2.0,4.0,189/QĐ-ĐHCNTT,2017-04-18 00:00:00,8.0,2013,1
...,...,...,...,...,...,...,...,...,...,...,...
1088,21278.0,9B23F85CXPvAibaEXe/QqI5At3DBgidRMGrBbipk,8.0,Xem xét chuyển qua hệ đào tạo không chính quy,2.0,6.0,244/QĐ-ĐHCNTT,NaT,12.0,2015,1
1089,21279.0,33B4B1F5XPvAibaEXe+4XPPXlKRz0EV4ZG7086cL,8.0,Xem xét chuyển qua hệ đào tạo không chính quy,2.0,6.0,244/QĐ-ĐHCNTT,NaT,12.0,2015,1
1090,21280.0,7ACF3993XPvAibaEXe9p78qxnH96y309RWXe+E33,8.0,Xem xét chuyển qua hệ đào tạo không chính quy,2.0,6.0,244/QĐ-ĐHCNTT,NaT,12.0,2015,1
1091,21281.0,7BEDC07CXPvAibaEXe9KWpGOh3zHl5qOD67ZCwQD,5.0,Xem xét chuyển qua hệ đào tạo không chính quy,2.0,6.0,252/QĐ-ĐHCNTT,NaT,12.0,2015,1


## diem

In [ ]:
df_transcript= pd.read_excel("sinhvien_dtb_hocky.xlsx")

In [ ]:
#normalize namhoc
df_transcript=df_transcript.merge(df_student[['mssv','namkhoahoc']], on='mssv')
df_transcript['namhoc']=df_transcript['namhoc']-df_transcript['namkhoahoc'].astype('int')+1
df_transcript['hk']=df_transcript['hocky']+(df_transcript['namhoc']-1)*2

In [ ]:
hk_xlhv = df_XLHV[df_XLHV['tinhtrang'].isin([5,7,8])][['mssv','hk']]

for indx, row in hk_xlhv.iterrows():
  df_transcript=df_transcript.drop(df_transcript.loc[((df_transcript['hk'] >= row['hk']) & (df_transcript['mssv']== row['mssv']))].index)

df_transcript=df_transcript.drop(df_transcript.loc[(df_transcript['hk'] >12)].index)

In [ ]:
#Thực hiện group_by để chuyển DataFrame về dạng wide data
df_transcript_gb=df_transcript.groupby(['mssv','hocky','namhoc','hk']).agg({'dtbhk': 'mean', 'sotchk': 'mean'})

In [ ]:
df_transcript_gb

dtbhk  sotchk
mssv                                     hocky namhoc hk               
0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G 1     1      1    8.84    17.0
                                               2      3    9.11    15.0
                                               3      5    8.54    20.0
                                               4      7    8.41     9.0
                                         2     1      2    9.00    21.0
...                                                         ...     ...
FFFA4234XPvAibaEXe+OhmjSj4XEzdTGgcTdT9fT 2     1      2    6.84    21.0
                                               2      4    7.86    19.0
                                               3      6    8.49    19.0
                                               4      8    8.10    13.0
                                         3     2      5    7.00     4.0

[68152 rows x 2 columns]

In [ ]:
pivot_df_transcript = df_transcript_gb.pivot_table(index='mssv', columns=['hk'], values=['dtbhk', 'sotchk'])

In [ ]:
pivot_df_transcript.head(3)

dtbhk                                 \
hk                                          1     2      3     4     5     6    
mssv                                                                            
0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G  8.84  9.00  9.110  8.75  8.54  8.74   
00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD  5.41  7.07  7.055  8.32  7.86  8.09   
000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q  8.85  8.64  8.410  8.05  7.46  7.28   

                                                               ... sotchk  \
hk                                          7     8    9   10  ...     3    
mssv                                                           ...          
0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G  8.41   NaN  NaN NaN  ...   15.0   
00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD  6.92  7.80  8.2 NaN  ...   15.5   
000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q  8.55  7.24  NaN NaN  ...   15.0   

                                                                              \
hk                                          4     5     6     7     8     9    
mssv                                                                           
0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G  19.0  20.0  20.0   9.0   NaN   NaN   
00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD  16.0  14.0  14.0  21.0  11.0  10.0   
000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q  20.0  20.0  19.0  15.0  10.0   NaN   

                                                      
hk                                        10  11  12  
mssv                                                  
0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G NaN NaN NaN  
00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD NaN NaN NaN  
000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q NaN NaN NaN  

[3 rows x 24 columns]

In [ ]:
# Tạo danh sách các cột mới
new_columns=[]
for column in pivot_df_transcript.columns:
    attrb, hocky = column
    if hocky>12:
      pivot_df_transcript = pivot_df_transcript.drop(columns=column)
      continue
    new_column = f"HK{hocky}_{attrb}"
    new_columns.append(new_column)

In [ ]:
pivot_df_transcript.columns=new_columns
pivot_df_transcript.reset_index(inplace=True)

In [ ]:
pivot_df_transcript.head(3)

,mssv,HK1_dtbhk,HK2_dtbhk,HK3_dtbhk,HK4_dtbhk,HK5_dtbhk,HK6_dtbhk,HK7_dtbhk,HK8_dtbhk,HK9_dtbhk,...,HK3_sotchk,HK4_sotchk,HK5_sotchk,HK6_sotchk,HK7_sotchk,HK8_sotchk,HK9_sotchk,HK10_sotchk,HK11_sotchk,HK12_sotchk
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,8.84,9.00,9.110,8.75,8.54,8.74,8.41,NaN,NaN,...,15.0,19.0,20.0,20.0,9.0,NaN,NaN,NaN,NaN,NaN
1,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,5.41,7.07,7.055,8.32,7.86,8.09,6.92,7.80,8.2,...,15.5,16.0,14.0,14.0,21.0,11.0,10.0,NaN,NaN,NaN
2,000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q,8.85,8.64,8.410,8.05,7.46,7.28,8.55,7.24,NaN,...,15.0,20.0,20.0,19.0,15.0,10.0,NaN,NaN,NaN,NaN


In [ ]:
df_transcript.to_excel('Preprocessed/sinhvien_dtb_hocky_preprocessed.xlsx', index=False)

## diemrl

In [ ]:
df_drl= pd.read_excel('diemrl.xlsx')
df_drl.head(3)

,id,mssv,lopsh,hocky,namhoc,drl,ghichu
0,1,2A10C466XPvAibaEXe8vcQUX+A6FeNIl5Loug1zV,ANTN2012,2,2013,81,NaN
1,2,903ADF1AXPvAibaEXe9p5G9u2qRxxZEjiJjD4Ztl,ANTN2012,2,2013,88,NaN
2,3,F06EDFFEXPvAibaEXe8KSegZW1rFPMNPBQ+c9rst,ANTN2012,2,2013,94,NaN


In [ ]:
#normalize namhoc
df_drl=df_drl.merge(df_student[['mssv','namkhoahoc']], on='mssv')
df_drl['namhoc']=df_drl['namhoc']-df_drl['namkhoahoc'].astype('int')+1

In [ ]:
df_drl[df_drl['namhoc']<1]

,id,mssv,lopsh,hocky,namhoc,drl,ghichu,namkhoahoc
1390,60284,31D5D488XPvAibaEXe85Kg8gbEhwbxD0x3mi2el8,NaN,1,0,61,NaN,2013
1391,62940,31D5D488XPvAibaEXe85Kg8gbEhwbxD0x3mi2el8,NaN,2,0,41,NaN,2013


In [ ]:
df_drl.loc[df_drl['mssv'].isin( pd.unique(df_drl[df_drl['namhoc']<1]['mssv'])), 'namhoc'] += 1
df_drl['hk']=df_drl['hocky']+(df_drl['namhoc']-1)*2

In [ ]:
hk_xlhv = df_XLHV[df_XLHV['tinhtrang'].isin([5,7,8])][['mssv','hk']]

for indx, row in hk_xlhv.iterrows():
  df_drl=df_drl.drop(df_drl.loc[((df_drl['hk'] >= row['hk']) & (df_drl['mssv']== row['mssv']))].index)

df_drl=df_drl.drop(df_drl.loc[df_drl['hk'] >12].index)

In [ ]:
df_drl

,id,mssv,lopsh,hocky,namhoc,drl,ghichu,namkhoahoc,hk
0,27,599DFFB8XPvAibaEXe879+AOg1gh8lJvChSN7o+V,ANTN2013,2,1,59,NaN,2013,2
1,19238,599DFFB8XPvAibaEXe879+AOg1gh8lJvChSN7o+V,ANTN2013,1,2,43,NaN,2013,3
2,26181,599DFFB8XPvAibaEXe879+AOg1gh8lJvChSN7o+V,KTPM2013,2,2,24,NaN,2013,4
3,37829,599DFFB8XPvAibaEXe879+AOg1gh8lJvChSN7o+V,KTPM2013,1,3,69,NaN,2013,5
4,49363,599DFFB8XPvAibaEXe879+AOg1gh8lJvChSN7o+V,KTPM2013,2,3,52,NaN,2013,6
...,...,...,...,...,...,...,...,...,...
69129,136059,7CD9B404XPvAibaEXe8M4ltS8XjC9c/g97NvpP52,TMCL2020,1,2,83,TMCL2020,2019,3
69130,140541,7CD9B404XPvAibaEXe8M4ltS8XjC9c/g97NvpP52,TMCL2020,2,2,79,NaN,2019,4
69131,144678,7CD9B404XPvAibaEXe8M4ltS8XjC9c/g97NvpP52,TMCL2020,1,3,100,NaN,2019,5
69132,159077,7CD9B404XPvAibaEXe8M4ltS8XjC9c/g97NvpP52,TMCL2020,2,3,83,NaN,2019,6


In [ ]:
pivot_df_drl=df_drl[['mssv','drl','hk']].pivot_table(index='mssv', columns='hk', values='drl', aggfunc='first')

new_columns=[]
for column in pivot_df_drl.columns:
    hocky = column
    if hocky>12:
      pivot_df_drl = pivot_df_drl.drop(columns=column)
      continue
    new_column = f"HK{hocky}_drl"
    new_columns.append(new_column)

pivot_df_drl.columns= new_columns
pivot_df_drl.reset_index(inplace=True)
pivot_df_drl.head(3)

,mssv,HK1_drl,HK2_drl,HK3_drl,HK4_drl,HK5_drl,HK6_drl,HK7_drl,HK8_drl,HK9_drl,HK10_drl,HK11_drl,HK12_drl
0,0001EB57XPvAibaEXe/twT+sf632fUXnsgPGeB4G,100.0,100.0,100.0,100.0,100.0,100.0,100.0,NaN,NaN,NaN,NaN,NaN
1,00046394XPvAibaEXe+fmxcqgvribEcT4YmJhSFD,81.0,76.0,75.0,91.0,77.0,71.0,64.0,70.0,76.0,70.0,NaN,NaN
2,000AD0D8XPvAibaEXe+RQyZpP6sq6qqIPZXybx3Q,85.0,100.0,96.0,100.0,100.0,90.0,86.0,84.0,58.0,NaN,NaN,NaN


In [ ]:
df_drl.to_excel('Preprocessed/diemrl_preprocessed.xlsx', index=False)

## Xeploaiav

In [ ]:
df_xeploaiav=pd.read_excel('04.xeploaiav.xlsx')
df_xeploaiav['mssv']= df_xeploaiav['mssv'].str.strip()

object_columns = df_xeploaiav.select_dtypes(include=['object']).columns.tolist()
df_xeploaiav[object_columns]=df_xeploaiav.select_dtypes(include=['object']).applymap(normalize_string)

df_xeploaiav = df_xeploaiav.dropna(how='all')
df_xeploaiav.columns = df_xeploaiav.columns.str.lstrip()

# Áp dụng hàm trên cho toàn bộ DataFrame
df_xeploaiav[object_columns]=df_xeploaiav.select_dtypes(include=['object']).applymap(replace_null)


<ipython-input-50-920a9dc1d597>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xeploaiav[object_columns]=df_xeploaiav.select_dtypes(include=['object']).applymap(replace_null)
<ipython-input-50-920a9dc1d597>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_xeploaiav[object_columns]=df_xeploaiav.select_dtypes(include=['object']).applymap(replace_null)
<ipython-input-50-920a9dc1d597>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_ind

In [ ]:
df_xeploaiav=df_xeploaiav.merge(df_student[['mssv','namkhoahoc', 'hedt']], on='mssv')

In [ ]:
def update_mamh(row):
    if row['namkhoahoc'] == '2016':
        if row['total'] < 26:
            return "Anh văn bổ túc"
        elif 26 <= row['total'] < 55:
            return "Anh văn 1"
        else:
            return "Anh văn 2"
        
    elif row['namkhoahoc'] == '2017':
        if row['total'] >= 96:
            return "Anh văn 3"
        elif 72 <= row['total'] < 96:
            return "Anh văn 2"
        elif 40 <= row['total'] < 72:
            return "Anh văn 1"
        else:
            return "Anh văn bổ túc"
        
    elif row['namkhoahoc'] in ['2018', '2019']:
        if row['total'] < 250:
            return "AVSC1"
        elif 250 <= row['total'] < 300:
            return "AVSC2"
        elif 300 <= row['total'] < 350:
            return "ENG01"
        elif 350 <= row['total'] < 400:
            return "ENG02"
        elif 400 <= row['total'] < 450:
            return "ENG03"
        elif 450 <= row['total'] < 500:
            return "ENG04"
        else:
            return "ENG05"
    else:
        return row['mamh'].strip()

df_xeploaiav['mamh'] = df_xeploaiav.apply(update_mamh, axis=1)

In [ ]:
pd.unique(df_xeploaiav['mamh'])

array(['Anh văn bổ túc', 'Anh văn 1', 'Anh văn 2', 'AVSC', 'Anh văn 3',
       'AVSC1', 'ENG01', 'AVSC2', 'ENG02', 'ENG05', 'ENG04', 'ENG03'],
      dtype=object)

In [ ]:
df_xeploaiav['pass_avsc']= ~df_xeploaiav['mamh'].isin(['Anh văn bổ túc', 'AVSC', 'AVSC1', 'AVSC2'])

In [ ]:
df_xeploaiav

,id,mssv,listening,reading,total,mamh,ghichu,listening,reading,mamh,ghichu,namkhoahoc,hedt,pass_avsc
0,1.0,DA75FFFEXPvAibaEXe8808q51BnmQhHT8REwWMQJ,NULL,NULL,15.0,Anh văn bổ túc,NULL,NaN,NaN,NaN,NaN,2016,CQUI,False
1,2.0,336B9F53XPvAibaEXe/jd8ghaf1GTNPSHNuhAHTZ,NULL,NULL,15.0,Anh văn bổ túc,NULL,NaN,NaN,NaN,NaN,2016,CQUI,False
2,3.0,67A51DC3XPvAibaEXe/7YVAxOsMwOtCFvKInEQ0e,NULL,NULL,16.0,Anh văn bổ túc,NULL,NaN,NaN,NaN,NaN,2016,CQUI,False
3,4.0,387E2EFEXPvAibaEXe96K1aB3B4cKSj8mKKciukN,NULL,NULL,16.0,Anh văn bổ túc,NULL,NaN,NaN,NaN,NaN,2016,CQUI,False
4,6.0,13D900A2XPvAibaEXe97wmuqTYuV7bJOKimbzKxM,NULL,NULL,18.0,Anh văn bổ túc,NULL,NaN,NaN,NaN,NaN,2016,CQUI,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4914,4962.0,5DED219EXPvAibaEXe+g4B5cywZjwcehNH9APFMj,21,15,195.0,AVSC1,Qui đổi TOEIC,21,15,AVSC1,Qui đổi TOEIC,2019,CQUI,False
4915,4963.0,794F4933XPvAibaEXe8YD29zP3MXDwVlPj1MfWns,17,21,205.0,AVSC1,Qui đổi TOEIC,17,21,AVSC1,Qui đổi TOEIC,2019,CLC,False
4916,4964.0,3BC1E78EXPvAibaEXe9zkQY6ucuvgG+vVB9Kue9g,28,32,325.0,ENG01,Qui đổi TOEIC,28,32,ENG01,Qui đổi TOEIC,2019,CTTT,True
4917,4965.0,EE2CD7AFXPvAibaEXe/kSDDr/jkTQyEWEYQF/mOJ,17,19,195.0,AVSC1,Qui đổi TOEIC,17,19,AVSC1,Qui đổi TOEIC,2019,CTTT,False


In [ ]:
df_xeploaiav.to_excel('Preprocessed/04.xeploaiav_preprocessed.xlsx', index=False)

## totnghiep

In [ ]:
df_totnghiep= pd.read_excel('14.totnghiep.xlsx')

In [ ]:
df_totnghiep['mssv']= df_totnghiep['mssv'].str.strip()

object_columns = df_totnghiep.select_dtypes(include=['object']).columns.tolist()
df_totnghiep[object_columns]=df_totnghiep.select_dtypes(include=['object']).applymap(normalize_string)

df_totnghiep = df_totnghiep.dropna(how='all')
df_totnghiep.columns = df_totnghiep.columns.str.lstrip()

# Áp dụng hàm trên cho toàn bộ DataFrame
df_totnghiep[object_columns]=df_totnghiep.select_dtypes(include=['object']).applymap(replace_null)

In [ ]:
df_totnghiep

,id,mssv,xeploai,soquyetdinh,ngaycapvb,xeploai,soquyetdinh,ngaycapvb
0,76328.0,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,Khá,178/QĐ_ĐHCNTT,14/04/2017,Khá,178/QĐ_ĐHCNTT,14/04/2017
1,76663.0,D0FE4969XPvAibaEXe/yXiKgsgy0slCmJ5EKt6Ki,Giỏi,713/QĐ_ĐHCNTT,06/10/2017,Giỏi,713/QĐ_ĐHCNTT,06/10/2017
2,76664.0,40F7E8D0XPvAibaEXe+nZBq3b0XEhfcwXLmoc4Pj,Giỏi,713/QĐ_ĐHCNTT,06/10/2017,Giỏi,713/QĐ_ĐHCNTT,06/10/2017
3,76665.0,0A049F45XPvAibaEXe9CtFSNMbfIz9qE7i0Fu4My,Khá,713/QĐ_ĐHCNTT,06/10/2017,Khá,713/QĐ_ĐHCNTT,06/10/2017
4,76666.0,7A7166DAXPvAibaEXe83V/kFvw2bBZ0o/KVHZc30,Khá,713/QĐ_ĐHCNTT,06/10/2017,Khá,713/QĐ_ĐHCNTT,06/10/2017
...,...,...,...,...,...,...,...,...
1842,79855.0,A2698E63XPvAibaEXe+D5C9Re7EGLdYoD7VxJkyw,Khá,354/QĐ-ĐHCNTT,09/6/2021,Khá,354/QĐ-ĐHCNTT,09/6/2021
1843,79856.0,B44C3B36XPvAibaEXe/ns/wSuHF6kEcjw3s5q9gL,Khá,354/QĐ-ĐHCNTT,09/6/2021,Khá,354/QĐ-ĐHCNTT,09/6/2021
1844,79857.0,175475BBXPvAibaEXe8dBqMaN7rGhkux+ZR35bi9,Khá,354/QĐ-ĐHCNTT,09/6/2021,Khá,354/QĐ-ĐHCNTT,09/6/2021
1845,79858.0,94A79F45XPvAibaEXe+dJypLAXKUXx9tmvinQDNg,Khá,354/QĐ-ĐHCNTT,09/6/2021,Khá,354/QĐ-ĐHCNTT,09/6/2021


In [ ]:
df_totnghiep=df_totnghiep.merge(df_student[['mssv','namkhoahoc']], on='mssv')

In [ ]:
df_totnghiep['namkhoahoc'].value_counts()

2013    563
2014    508
2015    458
2016    275
2017     41
Name: namkhoahoc, dtype: int64

In [ ]:
df_totnghiep=df_totnghiep[~df_totnghiep['mssv'].isin(df_BTH['mssv'])]

In [ ]:
df_totnghiep.to_excel('Preprocessed/14.totnghiep_preprocessed.xlsx', index=False)

In [ ]:
df_totnghiep['label']=0

In [ ]:
train_ratio = 0.7
# validation_ratio = 0.2
# test_ratio = 0.10

# train_neg, test_neg = train_test_split(df_totnghiep[['mssv','label']], test_size=1 - train_ratio)

# val_neg, test_neg = train_test_split(test_neg, test_size=test_ratio/(test_ratio + validation_ratio)) 
train_neg, test_neg = train_test_split(df_totnghiep[['mssv','label']], train_size=train_ratio, random_state= 18)

In [ ]:
test_neg

,mssv,label
1514,5066E750XPvAibaEXe+RTFcnyQdP1gV8o5sqlLTq,0
1789,F648358EXPvAibaEXe+NnQrnI1K9JIIcX9WCeXA+,0
376,5907F3FBXPvAibaEXe8ooYk4mkUGmFUd0afmTIV2,0
280,C63128F5XPvAibaEXe9Z67almz7y5mrP6fB5MbAb,0
1552,C9CD6314XPvAibaEXe8LX4hhYKrFZr9FZE839SqA,0
...,...,...
136,2D49AC3CXPvAibaEXe8Cym7B6GX+TjCt0Zgg9NLj,0
989,CF209E4FXPvAibaEXe+/EkvXK98KbbLh+JMUgKhh,0
968,04C9CC0EXPvAibaEXe/S5mChdnrLm80qIvJ4ryp4,0
969,6C1D614EXPvAibaEXe/nv6PZ4rt50gMq3SlqZ5wF,0


# EDA

In [ ]:
fig= go.Figure()
fig.add_trace(go.Bar(x= df_totnghiep.value_counts('namkhoahoc').index, y=  df_totnghiep.value_counts('namkhoahoc').values, name='tốt nghiệp'))
fig.add_trace(go.Bar(x= df_BTH.value_counts('namkhoahoc').index, y=  df_BTH.value_counts('namkhoahoc').values, name='Buộc thôi học'))
fig.update_layout(go.Layout(title='Thống kê số lượng sinh viên tốt nghiệp và bị buộc thôi học ở mỗi niên khóa', xaxis_title='Khóa học', yaxis_title='số lượng'))

In [ ]:
df_transcript_subset_1=df_transcript.merge(df_totnghiep[['mssv']], on= 'mssv')
df_transcript_subset_2=df_transcript.merge(df_BTH[['mssv']], on= 'mssv')

# Tạo histogram cho từng nhãn
histogram1= go.Histogram(x=df_transcript_subset_1['dtbhk'], name='TN', opacity=0.7)

histogram2 = go.Histogram(x=df_transcript_subset_2['dtbhk'], name='BTH', opacity=0.7)
fig = go.Figure(data=[histogram1, histogram2])
fig.update_layout(
    title_text='Phân bố điểm giữa sinh viên đã tốt nghiệp và sinh viên bị buộc thôi học',
    xaxis=dict(title='Điểm'),
    yaxis=dict(title='Tần số'),
    bargap=0.1,
    barmode='overlay'
)

# Hiển thị đồ thị
fig.show()


In [ ]:
df_drl_subset_1=df_drl.merge(df_totnghiep[['mssv']], on= 'mssv')
df_drl_subset_2=df_drl.merge(df_BTH[['mssv']], on= 'mssv')

# Tạo histogram cho từng nhãn
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_drl_subset_1['drl'], name='TN', opacity=0.7))
fig.add_trace(go.Histogram(x=df_drl_subset_2['drl'], name='BTH', opacity=0.7))

fig.update_layout(
    title_text='Phân bố đrl giữa sinh viên đã tốt nghiệp và sinh viên bị buộc thôi học',
    xaxis=dict(title='Điểm'),
    yaxis=dict(title='Tần số'),
    bargap=0.4,
    barmode='overlay'
)

# Hiển thị đồ thị
fig.show()


In [ ]:
merged1_dataframe = pd.merge(df_xeploaiav[['mssv', 'pass_avsc']], df_totnghiep[['mssv', 'label']], on='mssv', how='right')

In [ ]:
merged2_dataframe = pd.merge(df_xeploaiav[['mssv', 'pass_avsc']], df_BTH[['mssv', 'label']], on='mssv', how='right')

In [ ]:
EDA_dataframe = pd.concat([merged1_dataframe, merged2_dataframe])

In [ ]:
EDA_dataframe

,mssv,pass_avsc,label
0,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,NaN,0
1,D0FE4969XPvAibaEXe/yXiKgsgy0slCmJ5EKt6Ki,NaN,0
2,40F7E8D0XPvAibaEXe+nZBq3b0XEhfcwXLmoc4Pj,NaN,0
3,0A049F45XPvAibaEXe9CtFSNMbfIz9qE7i0Fu4My,NaN,0
4,7A7166DAXPvAibaEXe83V/kFvw2bBZ0o/KVHZc30,NaN,0
...,...,...,...
1088,9B23F85CXPvAibaEXe/QqI5At3DBgidRMGrBbipk,NaN,1
1089,33B4B1F5XPvAibaEXe+4XPPXlKRz0EV4ZG7086cL,NaN,1
1090,7ACF3993XPvAibaEXe9p78qxnH96y309RWXe+E33,NaN,1
1091,7BEDC07CXPvAibaEXe9KWpGOh3zHl5qOD67ZCwQD,NaN,1


In [ ]:
EDA_dataframe_temp = pd.DataFrame({'pass_avsc': EDA_dataframe['pass_avsc'].map({True: 'Pass', False: 'Fail', np.nan:'Unknow'}),
                        'label': EDA_dataframe['label'].map({0: 'Tốt nghiệp', 1: 'BTH'})})

correlation = EDA_dataframe_temp.groupby(['pass_avsc', 'label']).size().unstack().fillna(0)

# Tạo biểu đồ multi_bar
fig = go.Figure()

for column in reversed(correlation.columns):
    fig.add_trace(go.Bar(name=column, x=correlation.index, y=correlation[column]))

# Đặt tiêu đề cho biểu đồ và trục x, y
fig.update_layout(title_text='Biểu đồ Tương quan giữa yếu tố xếp lớp AV đầu vào trên AVCS và sinh viên bị buộc tôi học hoặc đã tốt nghiệp',
                  xaxis_title='pass_avsc',
                  yaxis_title='Số lượng')

# Hiển thị biểu đồ
fig.show()

# Create Dataset

In [ ]:
train_set=pd.concat([train_pos, train_neg])
test_set= pd.concat([test_pos, test_neg])
#val_set= pd.concat([val_pos, val_neg])

data=pivot_df_drl.merge(pivot_df_transcript, on='mssv', how='right')
data=data.merge(df_xeploaiav[['mssv','pass_avsc']], how='left', on='mssv')

In [ ]:
dataset= pd.concat([train_set,test_set])
fig = go.Figure(data=[go.Pie(labels=dataset['label'].value_counts().index, values=dataset['label'].value_counts().values, textinfo='value+percent', hole=0.5)])
fig.update_layout(go.Layout(title='Thống kê số lượng quan sát ở mỗi class'))
fig.update_layout(width=800, height=600)
fig.show()

In [ ]:
def generate_data(input_set, backward=1):
  df_warning_subset=input_set.merge(df_warning, on='mssv')
  output_data=input_set.merge(data, on='mssv')
  if backward >0:
    for i in range(backward):
      checkpoint = pd.DataFrame(df_transcript.groupby('mssv')['hk'].max()) - i
      checkpoint.reset_index(inplace=True)
      checkpoint.loc[checkpoint['hk']>12, 'hk']=12
      checkpoint.loc[checkpoint['hk']<1, 'hk']=1
      
      for mssv, hk in checkpoint.itertuples(index=False):
        col_name = [f'HK{hk}_drl', f'HK{hk}_sotchk', f'HK{hk}_dtbhk']
        output_data.loc[output_data['mssv'] == mssv, col_name] = np.nan
        output_data.dropna(thresh = 2, subset=output_data.columns[2:-1], inplace=True)

      dtb_columns = [col for col in output_data.columns if col.endswith('_dtbhk')]  
      nan_counts = output_data[dtb_columns].isna().sum(axis=1)
      drop_mask = nan_counts > 10

      output_data = output_data.drop(output_data[drop_mask].index)

      for mssv, hk in checkpoint.itertuples(index=False):
        df_warning_subset.drop(df_warning_subset[(df_warning_subset['mssv'] == mssv) & (df_warning_subset['hk'] == hk)].index, inplace=True)

  # checkpoint_hk = pd.DataFrame(df_transcript.groupby('mssv')['hk'].max()) - backward 
  # checkpoint_hk.loc[checkpoint_hk['hk']>12, 'hk']=12
  # checkpoint_hk.loc[checkpoint_hk['hk']<1, 'hk']=1
  # output_data = output_data.merge(checkpoint_hk, on='mssv', how='left')

  pivot_df_warning_subset=df_warning_subset[['mssv','lanthu','lydo']].pivot_table(index='mssv', columns='lanthu', values='lydo', aggfunc='first')
  while (True):
    if 6 not in pivot_df_warning_subset.columns:
      pivot_df_warning_subset[max(pivot_df_warning_subset.columns)+1]=np.nan
    else: break

  new_columns=[]
  for column in pivot_df_warning_subset.columns:
    lanthu = column
    new_column = f"CCHV_{lanthu}"
    new_columns.append(new_column)

  pivot_df_warning_subset.columns= new_columns
  pivot_df_warning_subset.reset_index(inplace=True)
  return output_data.merge(pivot_df_warning_subset, on='mssv', how='left').drop('mssv', axis=1)


In [ ]:
train_data = generate_data(train_set, backward=1).sample(frac = 1)
#val_data_1 = generate_data(val_set, backward=1)
#val_data_2 = generate_data(val_set, backward=2)
#val_data= pd.concat([val_data_1, val_data_2]).sample(frac = 1)
test_data_1 =  generate_data(test_set, backward=1).sample(frac = 1)
test_data_2 =  generate_data(test_set, backward=2).sample(frac = 1)



In [ ]:
train_data.head(5)

,label,HK1_drl,HK2_drl,HK3_drl,HK4_drl,HK5_drl,HK6_drl,HK7_drl,HK8_drl,HK9_drl,...,HK10_sotchk,HK11_sotchk,HK12_sotchk,pass_avsc,CCHV_1,CCHV_2,CCHV_3,CCHV_4,CCHV_5,CCHV_6
613,1,56.0,56.0,56.0,36.0,56.0,49.0,60.0,NaN,45.0,...,NaN,NaN,NaN,NaN,Bị cảnh cáo vì đtb 2 học kỳ liên tiếp < 4,NaN,NaN,NaN,NaN,NaN
1271,0,80.0,80.0,87.0,65.0,40.0,60.0,NaN,70.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,1,79.0,65.0,80.0,80.0,81.0,75.0,86.0,79.0,58.0,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
608,1,89.0,88.0,46.0,80.0,78.0,74.0,64.0,78.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
895,0,64.0,71.0,80.0,80.0,80.0,85.0,74.0,NaN,NaN,...,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
train_data.to_excel('Preprocessed/train.xlsx', index=False)
#val_data.to_excel('Preprocessed/val.xlsx', index=False)
test_data_1.to_excel('Preprocessed/test_1.xlsx', index=False)
test_data_2.to_excel('Preprocessed/test_2.xlsx', index=False)